# 学习使用SQLAlchemy ORM框架

安装框架

In [1]:
! pip install sqlalchemy


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python -m pip install --upgrade pip


## 创建引擎

SQLAlchemy 通过不同的引擎连接到不同的数据库实例上。

其最主要的参数是连接字符串，"sqlite+pysqlite:///:memory:"
它由3段组成：

sqlite:
pysqlite:
/:memory:


In [5]:
from sqlalchemy import create_engine

engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

准备好发动机对象后，我们现在可以继续研究引擎的基本操作及其主要的交互式端点，即连接和结果。我们还将为这些对象（称为会话）介绍ORM的外墙。

> 使用 ORM 时，引擎由另一个称为会话的对象管理。 
> 现代 SQLAlchemy 中的 Session 强调事务和 SQL 执行模式，这与下面讨论的 Connection 在很大程度上相同，因此虽然本小节是以核心为中心的，但这里的所有概念本质上也与 ORM 使用相关，并推荐用于 所有 ORM 学习者。 
> Connection 使用的执行模式将在本节末尾与 Session 的执行模式进行对比。

在下面使用text构造器执行了一条查询命令命令，用以打印 hello world

首先从engine中获取一个连接，并用上下文管理器管理这个连接以保证在连接在被用完后会被正常关闭。
然后使用连接执行 text构造的SQL语句。

In [3]:
from sqlalchemy import text
with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())

2023-05-31 06:56:41,394 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-31 06:56:41,395 INFO sqlalchemy.engine.Engine select 'hello world'
2023-05-31 06:56:41,396 INFO sqlalchemy.engine.Engine [generated in 0.00183s] ()
[('hello world',)]
2023-05-31 06:56:41,398 INFO sqlalchemy.engine.Engine ROLLBACK


连接中的事务并不会自动提交，从上文中的结果可以发现，最后的Engine ROLLBACK，如果一个连接没有提交，那么会在连接关闭前回滚。

## 向数据库提交更改
我们可以改变上面的示例来创建一个表并插入一些数据，然后使用 Connection.commit() 方法提交事务，该方法在我们获取 Connection 对象的块内调用：
如下：
将创建一个数据表并向其中插入数据。

In [7]:
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}],
    )
    conn.commit()

2023-05-31 07:05:57,792 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-31 07:05:57,793 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2023-05-31 07:05:57,795 INFO sqlalchemy.engine.Engine [generated in 0.00331s] ()
2023-05-31 07:05:57,796 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2023-05-31 07:05:57,797 INFO sqlalchemy.engine.Engine [generated in 0.00102s] [(1, 1), (2, 4)]
2023-05-31 07:05:57,798 INFO sqlalchemy.engine.Engine COMMIT


engine.connect() 是需要手动提交数据到数据库；

还有另一种合作数据样式，即我们可以将“连接”块声明为前面的连接块。对于这种操作模式，我们使用Engine.begin（）方法来获取连接，而不是eNgire.connect（）方法。此方法既可以管理连接的范围，又要将所有内容包装在连接中，并在结束时，假设成功的块，或者在例外增加的情况下回滚。这种样式可以称为开始一次：

engine.begin() 将会在结束时自动提交。

In [9]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}],
    )

2023-05-31 07:06:51,342 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-31 07:06:51,343 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2023-05-31 07:06:51,344 INFO sqlalchemy.engine.Engine [cached since 53.55s ago] [(6, 8), (9, 10)]
2023-05-31 07:06:51,345 INFO sqlalchemy.engine.Engine COMMIT


## 从数据库查询内容

In [10]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    print('Type:', type(result))
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2023-05-31 07:17:11,126 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-31 07:17:11,127 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2023-05-31 07:17:11,129 INFO sqlalchemy.engine.Engine [cached since 719.2s ago] ()
Type: <class 'sqlalchemy.engine.cursor.CursorResult'>
x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2023-05-31 07:17:11,131 INFO sqlalchemy.engine.Engine ROLLBACK


上面，我们执行的“选择”字符串从表中选择了所有行。返回的对象称为结果，代表结果行的一个可迭代的对象。

结果有许多用于获取和转换行的方法，例如result.all（）方法，以返回所有行对象的列表。它还实现了Python迭代器接口，因此我们可以直接迭代行的集合。

**元组分配** - 这是最PYTHON-偶像的样式，它是在接收到每行的位置上分配变量：
```python
result = conn.execute(text("select x, y from some_table"))

for x, y in result:
    ...
```

**索引分配** - 元组是Python序列，因此也可以使用常规整数访问：
```python
result = conn.execute(text("select x, y from some_table"))

for row in result:
    x = row[0]
```

**属性名字** - 由于这些是python命名为元组，因此该元组具有与每列名称匹配的动态属性名称。这些名称通常是SQL语句分配给每行列的名称。尽管它们通常是可预测的，并且也可以由标签控制，但在较不明显的情况下，它们可能受到数据库特定行为的影响：

```python
result = conn.execute(text("select x, y from some_table"))

for row in result:
    y = row.y

    # illustrate use with Python f-strings
    print(f"Row: {row.x} {y}")
```

**字典访问** - 要将行作为 Python 映射对象接收，这本质上是 Python 到通用 dict 对象的接口的只读版本，可以使用 Result.mappings() 修饰符将 Result 转换为 MappingResult 对象； 这是一个结果对象，它产生类似字典的 RowMapping 对象而不是 Row 对象：
```python
result = conn.execute(text("select x, y from some_table"))

for dict_row in result.mappings():
    x = dict_row["x"]
    y = dict_row["y"]
```



## 发送查询参数

conn.execute() 接受一些参数。


In [11]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table WHERE y > :y"), {"y": 2})
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2023-05-31 07:34:47,510 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-31 07:34:47,512 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ?
2023-05-31 07:34:47,513 INFO sqlalchemy.engine.Engine [generated in 0.00353s] (2,)
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2023-05-31 07:34:47,515 INFO sqlalchemy.engine.Engine ROLLBACK


## Sending Multiple Parameters -- executemany

In [ ]:
with engine.connect() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 11, "y": 12}, {"x": 13, "y": 14}],
    )
    conn.commit()

## ORM Session


In [12]:
from sqlalchemy.orm import Session

stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y")
with Session(engine) as session:
    result = session.execute(stmt, {"y": 6})
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2023-05-31 08:32:33,896 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-31 08:32:33,897 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ? ORDER BY x, y
2023-05-31 08:32:33,899 INFO sqlalchemy.engine.Engine [generated in 0.00188s] (6,)
x: 6  y: 8
x: 9  y: 10
2023-05-31 08:32:33,901 INFO sqlalchemy.engine.Engine ROLLBACK


In [13]:
with Session(engine) as session:
    result = session.execute(
        text("UPDATE some_table SET y=:y WHERE x=:x"),
        [{"x": 9, "y": 11}, {"x": 13, "y": 15}],
    )
    session.commit()

2023-05-31 08:32:48,509 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-31 08:32:48,510 INFO sqlalchemy.engine.Engine UPDATE some_table SET y=? WHERE x=?
2023-05-31 08:32:48,512 INFO sqlalchemy.engine.Engine [generated in 0.00123s] [(11, 9), (15, 13)]
2023-05-31 08:32:48,512 INFO sqlalchemy.engine.Engine COMMIT
